# Feature Extraction

In [ ]:
!pip install pyradiomics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 10.8 MB/s 
     |████████████████████████████████| 52.8 MB 183 kB/s 
     |████████████████████████████████| 109 kB 43.2 MB/s 
     |████████████████████████████████| 500 kB 52.1 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=9f15a36543b9f68964ec9bc6524e43611099557036fbd8f86b7ed8fb76b81695
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
import radiomics
from radiomics import featureextractor, firstorder, glcm, imageoperations, shape, glrlm, glszm
import six

from mpl_toolkits.axes_grid1 import make_axes_locatable
import SimpleITK as sitk
import matplotlib.pyplot as plt
import os
import numpy
import pandas as pd

## 1. Connect Colab to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [ ]:
image_dir = '/content/drive/MyDrive/Colab Notebooks/nnUNet_raw_data_base/nnUNet_raw_data/T2_test_data/imagesTs'
label_dir = '/content/drive/MyDrive/Colab Notebooks/nnUNet_Results_Folder/T2_test_data/predTs'
results_dir = '/content/drive/MyDrive/Test Data/'

## 2. Set Feature Extraction Setting

In [ ]:
# shape feature
ShapeFeaturesExtractor = featureextractor.RadiomicsFeatureExtractor()
ShapeFeaturesExtractor.disableAllFeatures()
ShapeFeaturesExtractor.enableFeatureClassByName('shape')

# first order features
firstOrderFeaturesExtractor = featureextractor.RadiomicsFeatureExtractor(binCount=128, verbose=True, interpolator=None)
firstOrderFeaturesExtractor.disableAllFeatures()
firstOrderFeaturesExtractor.enableFeatureClassByName('firstorder')

# glcm features
glcmFeaturesExtractor = featureextractor.RadiomicsFeatureExtractor(binCount=128, verbose=True, interpolator=None)
glcmFeaturesExtractor.disableAllFeatures()
glcmFeaturesExtractor.enableFeatureClassByName('glcm')

# iszm features
iszmFeaturesExtractor = featureextractor.RadiomicsFeatureExtractor(binCount=32, verbose=True, interpolator=None)
iszmFeaturesExtractor.disableAllFeatures()
iszmFeaturesExtractor.enableFeatureClassByName('glszm')

INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'la

## 4. Derive Features from Images

In [ ]:
maxPatNum = 40
ShapeFeaturesStorage = numpy.zeros((maxPatNum, 14))

FeatureStorage = numpy.zeros((maxPatNum, 18 + 24 + 16))  # Hist + GLCM + ISZM + GLRLM

In [ ]:
patNum = 1
while patNum <= maxPatNum:

    print(" ")
    print("[patNum "+str(patNum)+"] Processing Start...")

    try:
        # Load images
        print("Load image files...")

        if patNum < 10:
          imageName = os.path.join(image_dir, "LIVER_00" + str(patNum) + "_0000.nii.gz")
          ROIName = os.path.join(label_dir, "LIVER_00" + str(patNum) + ".nii.gz")

        else:
          imageName = os.path.join(image_dir, "LIVER_0" + str(patNum) + "_0000.nii.gz")
          ROIName = os.path.join(label_dir, "LIVER_0" + str(patNum) + ".nii.gz")

        image = sitk.ReadImage(imageName)
        ROI = sitk.ReadImage(ROIName)
        
        ROI.CopyInformation(image)

        # 3D shape based feature
        # Feature n = 14
        print("Calculating shape features...")
        ShapeFeatures = ShapeFeaturesExtractor.execute(image, ROI, label=3) # label 3 refers to liver

        # Histogram Based Features, LoG off
        # Feature n = 18
        print("Calculating histogram features...")
        HistFeatures = firstOrderFeaturesExtractor.execute(image, ROI, label=3)

        # GLCM Based Features, LoG OFF
        # Feature n = 24
        print("Calculating GLCM features...")
        GLCMFeatures = glcmFeaturesExtractor.execute(image, ROI, label=3)

        # ISZM Based Features, LoG OFF
        # Feature n = 16
        print("Calculating ISZM features...")
        ISZMFeatures = iszmFeaturesExtractor.execute(image, ROI, label=3)

        # Save features in storage variable
        ShapeFeaturesStorage[patNum - 1, ] = [ShapeFeatures[x] for x in list(
            filter(lambda k: k.startswith("original_") or k.startswith("log"), ShapeFeatures))]

        FeatureStorage[patNum - 1, ] = \
            [HistFeatures[x] for x in list(filter(lambda k: k.startswith("original_"), HistFeatures))] \
            + [GLCMFeatures[x] for x in list(filter(lambda k: k.startswith("original_"), GLCMFeatures))] \
            + [ISZMFeatures[x] for x in list(filter(lambda k: k.startswith("original_"), ISZMFeatures))]

    except:
        print("[FATAL Error] Something Wrong... patNum " + str(patNum))
        patNum += 1
        continue;

    patNum += 1

 
[patNum 1] Processing Start...
Load image files...


INFO:radiomics.featureextractor:Calculating features with label: 3
INFO:radiomics.featureextractor:Loading image and mask


Calculating shape features...


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Calculating features with label: 3
INFO:radiomics.featureextractor:Loading image and mask


Calculating histogram features...


...

INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Calculating features with label: 3
INFO:radiomics.featureextractor:Loading image and mask


Calculating GLCM features...


INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Calculating features with label: 3
INFO:radiomics.featureextractor:Loading image and mask


Calculating ISZM features...


INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing glszm


In [ ]:
# Total Feature values
TotalFeatures = numpy.concatenate((ShapeFeaturesStorage, FeatureStorage), axis=1)

# Feature names
ShapeFeatureNames = list(filter(lambda k: k.startswith("original_"), ShapeFeatures))
HistFeatureNames = list(filter(lambda k: k.startswith("original_"), HistFeatures))
GLCMFeatureNames = list(filter(lambda k: k.startswith("original_"), GLCMFeatures))
ISZMFeatureNames = list(filter(lambda k: k.startswith("original_"), ISZMFeatures))

# Pre
FeatureNames = numpy.concatenate(
    ([s for s in HistFeatureNames], 
     [s for s in GLCMFeatureNames],
     [s for s in ISZMFeatureNames]), 
     axis=0)

FeatureNames = numpy.concatenate((ShapeFeatureNames, FeatureNames), axis=0)

In [ ]:
# save results
RadiomicFeatures = pd.DataFrame(TotalFeatures, columns=FeatureNames)

# replace 0s with NAs
RadiomicFeatures.replace(0, numpy.na, inplace=True)

# write .csv file
RadiomicFeatures.to_csv(os.path.join(results_dir, 'RadiomicFeatures.csv'), index=False, header=True, sep=',')